# Data Collection

In order to begin analyzing stocks we first need to collect data to analyze. We will do this by scraping a specified timeframe of stock information for a specified number of stocks.


## Getting Started
To get started, we must import some libraries:

In [1]:
import sys
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
print("======================")


import yfinance as yf
import pandas as pd

print ("Successfully imported libraries")

Successfully imported libraries


We will begin by specifying a list of tickers we want to scrape:

In [2]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

From here, we have to define some options:

- Interval: This represents the frequency we want to get stock data
- Time Frame (2 options):
    - A)
        - Period: This represents a timeframe in english (1m, 5m, 1d, 1y, ...)
    - B)
        - Start Date: This is the starting date for data scraping
        - End Date: This is the ending date for data scraping

In [3]:
period = '10y'
start_date = None
end_date = None
interval = "1d"

Just a quick check to ensure variables are instantiated correctly:

In [4]:
if tickers == None:
    raise Exception("You must specify a list of tickers to scrape")

if period == None and start_date == None and end_date == None:
            raise Exception("You must specify one timeframe in order to scrape")
        
if period == None and (start_date == None or end_date == None):
    raise Exception("You must specify both ends of the timeframe in order to scrape")

if period not in ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']:
    raise Exception("Please input a valid period")

if interval not in ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']:
    raise Exception("Please input a valid time interval")

if period != None and (start_date != None or end_date != None):
    raise Exception("You can only specify one type of timeframe in order to scrape")

print("Passed!")

Passed!


## Scraping Data
Now that we have defined the scraping parameters, we can actually begin to scrape. We do this by calling the YFinance download method for each stock. However, we have to consider two cases:
- We are using period
- We are using start/end dates


In [5]:
ticker_string = ' '.join(tickers) #convert from list to space-separated string
print(f"Ticker String: \n{ticker_string}")

if period != None: #if using period
    data = yf.download(
        ticker_string,
        period = period,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )
else:
    data = yf.download( #if using start/end dates
        ticker_string,
        start = start_date, 
        end = end_date,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )


data = data.drop([(i, 'Close') for i in tickers], axis=1)
data = data.rename({"Adj Close": "Close"}, axis=1)


data.head()

Ticker String: 
AMZN TSLA BABA HD TM NKE MCD LOW SBUX JD PDD BKNG GM MELI TJX NIO F LULU HMC CMG
[*********************100%***********************]  20 of 20 completed


HMC                                            TSLA         \
                 Open       High        Low      Close  Volume   Open   High   
Date                                                                           
2011-11-10  29.480000  29.490000  29.240000  29.200457  341800  6.188  6.300   
2011-11-11  29.520000  29.709999  29.520000  29.508982  650400  6.380  6.900   
2011-11-14  29.480000  29.650000  29.410000  29.429361  256300  6.600  6.708   
2011-11-15  29.450001  29.680000  29.270000  29.399506  208700  6.584  6.880   
2011-11-16  29.200001  29.219999  28.780001  28.643120  441600  6.696  7.000   

                                    ...        NKE                        \
              Low  Close    Volume  ...       Open       High        Low   
Date                                ...                                    
2011-11-10  6.130  6.266   3736500  ...  23.355000  23.635000  23.142500   
2011-11-11  6.114  6.728  19341500  ...  23.687500  24.102501  23.612499   
2011-11-14  6.524  6.644   6628500  ...  23.959999  24.222500  23.799999   
2011-11-15  6.546  6.786   4455000  ...  23.815001  24.150000  23.667500   
2011-11-16  6.680  6.988   9166000  ...  23.785000  23.875000  23.385000   

                                       GM                                   \
                Close   Volume       Open       High        Low      Close   
Date                                                                         
2011-11-10  20.891666  9393600  22.440001  22.850000  21.930000  17.544685   
2011-11-11  21.387714  8108000  22.950001  23.100000  22.219999  17.397837   
2011-11-14  21.298742  6849200  22.559999  23.290001  22.510000  17.768824   
2011-11-15  21.343222  7013600  22.809999  23.530001  22.600000  18.047068   
2011-11-16  20.842728  8850000  23.240000  23.350000  22.620001  17.506048   

                      
              Volume  
Date                  
2011-11-10  15866600  
2011-11-11  15438300  
2011-11-14  13433300  
2011-11-15  13867200  
2011-11-16   9297200  

[5 rows x 100 columns]

# Adding Technical Indicators

Using the [TA-Lib](https://mrjbq7.github.io/ta-lib/) package, we can broadcast technical indicators across the time series into their own dataframe columns

In [6]:
import talib
!{sys.executable} -m pip install TA-Lib

def computeRSI(df):
    return talib.RSI(df["Close"], timeperiod=14)

def computeUltimateOscillator(df):
    return talib.ULTOSC(df["High"], df["Low"], df["Close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

def computeBollingerBands(df):
    upperband, middleband, _ = talib.BBANDS(df["Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    return (upperband - middleband) / df["Close"] # need to normalize so value is not dependent on share price

def computeChaikinOscillator(df):
    return talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"], fastperiod=3, slowperiod=10)

def computeNATR(df):
    return talib.NATR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeSMA(df, window):
    return talib.SMA(df["Close"], timeperiod=window) / df["Close"] # need to normalize so value is not dependent on share price

def computeSAR(df):
    return talib.SAR(df["High"], df["Low"], acceleration=0, maximum=0) / df["Close"] # need to normalize so value is not dependent on share price

def computeWilliamsR(df):
    return talib.WILLR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeAPO(df):
    return talib.APO(df["Close"], fastperiod=12, slowperiod=26, matype=0) / df["Close"]# need to normalize so value is not dependent on share price

def computeROC(df, window):
    return talib.ROCP(df["Close"], timeperiod=window) 


Now, we can apply these functions across the collected timeseries

In [7]:
for ticker in tickers:
    df = data[ticker]
    data[ticker, 'RSI'] = computeRSI(df)
    data[ticker, 'Ultimate'] = computeUltimateOscillator(df)
    data[ticker, 'BandRadius'] = computeBollingerBands(df)
    data[ticker, 'Chaikin'] = computeChaikinOscillator(df)
    data[ticker, 'NATR'] = computeNATR(df)
    data[ticker, 'SMA_5'] = computeSMA(df, 5)
    data[ticker, 'SMA_20'] = computeSMA(df, 20)
    data[ticker, 'SMA_100'] = computeSMA(df, 100)
    data[ticker, 'SAR'] = computeSAR(df)
    data[ticker, 'Williams%R'] = computeWilliamsR(df)
    data[ticker, 'APO'] = computeAPO(df)
    data[ticker, 'ROC_5'] = computeROC(df, 5)
    data[ticker, 'ROC_20'] = computeROC(df, 20)
    data[ticker, 'ROC_100'] = computeROC(df, 100)

data.head(100)

HMC                                            TSLA         \
                 Open       High        Low      Close  Volume   Open   High   
Date                                                                           
2011-11-10  29.480000  29.490000  29.240000  29.200457  341800  6.188  6.300   
2011-11-11  29.520000  29.709999  29.520000  29.508982  650400  6.380  6.900   
2011-11-14  29.480000  29.650000  29.410000  29.429361  256300  6.600  6.708   
2011-11-15  29.450001  29.680000  29.270000  29.399506  208700  6.584  6.880   
2011-11-16  29.200001  29.219999  28.780001  28.643120  441600  6.696  7.000   
...               ...        ...        ...        ...     ...    ...    ...   
2012-03-29  38.270000  38.320000  38.040001  38.058128  233600  7.638  7.638   
2012-03-30  38.509998  38.509998  38.250000  38.247223  240700  7.504  7.588   
2012-04-02  38.439999  38.919998  38.410000  38.655273  298300  7.466  7.594   
2012-04-03  38.959999  38.959999  38.169998  38.207413  604200  7.340  7.694   
2012-04-04  37.400002  37.400002  36.950001  37.142502  588900  7.054  7.098   

                                    ...       CMG                      \
              Low  Close    Volume  ...      NATR     SMA_5    SMA_20   
Date                                ...                                 
2011-11-10  6.130  6.266   3736500  ...       NaN       NaN       NaN   
2011-11-11  6.114  6.728  19341500  ...       NaN       NaN       NaN   
2011-11-14  6.524  6.644   6628500  ...       NaN       NaN       NaN   
2011-11-15  6.546  6.786   4455000  ...       NaN       NaN       NaN   
2011-11-16  6.680  6.988   9166000  ...       NaN  1.028869       NaN   
...           ...    ...       ...  ...       ...       ...       ...   
2012-03-29  7.406  7.466   3982000  ...  1.436964  0.999615  0.967851   
2012-03-30  7.336  7.448   4432000  ...  1.459570  1.007072  0.976742   
2012-04-02  7.306  7.316   5143000  ...  1.488375  1.003098  0.978846   
2012-04-03  7.334  7.602   5490500  ...  1.451470  0.994349  0.974037   
2012-04-04  6.938  7.000  22409000  ...  1.555464  1.005333  0.989219   

                                                                         \
             SMA_100       SAR Williams%R       APO     ROC_5    ROC_20   
Date                                                                      
2011-11-10       NaN       NaN        NaN       NaN       NaN       NaN   
2011-11-11       NaN  0.977441        NaN       NaN       NaN       NaN   
2011-11-14       NaN  0.992355        NaN       NaN       NaN       NaN   
2011-11-15       NaN  1.021010        NaN       NaN       NaN       NaN   
2011-11-16       NaN  1.055799        NaN       NaN       NaN       NaN   
...              ...       ...        ...       ...       ...       ...   
2012-03-29       NaN  0.713233 -19.249471  0.028483  0.005955  0.067293   
2012-03-30       NaN  0.717703 -28.443432  0.029380  0.005993  0.059865   
2012-04-02       NaN  0.717017 -28.250379  0.028758 -0.014834  0.064685   
2012-04-03       NaN  0.710463 -15.974781  0.027696  0.002112  0.090266   
2012-04-04  0.869912  0.719355 -45.294653  0.026353 -0.007260  0.063606   

                    
           ROC_100  
Date                
2011-11-10     NaN  
2011-11-11     NaN  
2011-11-14     NaN  
2011-11-15     NaN  
2011-11-16     NaN  
...            ...  
2012-03-29     NaN  
2012-03-30     NaN  
2012-04-02     NaN  
2012-04-03     NaN  
2012-04-04     NaN  

[100 rows x 380 columns]

At this point, we have a 3D nested dataframe, split by tickers and then by columns. To put this data in CSV files, we will create a CSV file for each ticker containing the relevant information.

In [8]:
import os

for ticker in tickers:
    print(f"Saving data for {ticker}")
    
    tickerdata = data[ticker].dropna() #remove nan values
    if not os.path.exists("data"):
        os.mkdir("data")
    tickerdata.to_csv(f'./data/{ticker}.csv') #write to csv file

print("======================")
print("Saved stocks to CSV files. Take a look inside /data for them.")

Saving data for AMZN
Saving data for TSLA
Saving data for BABA
Saving data for HD
Saving data for TM
Saving data for NKE
Saving data for MCD
Saving data for LOW
Saving data for SBUX
Saving data for JD
Saving data for PDD
Saving data for BKNG
Saving data for GM
Saving data for MELI
Saving data for TJX
Saving data for NIO
Saving data for F
Saving data for LULU
Saving data for HMC
Saving data for CMG
Saved stocks to CSV files. Take a look inside /data for them.
